# **Anomalous Flight Detection Using Open Source Data**

## Extracting Files ##

Only run this block if you are loading the raw opensky data

In [ ]:
import pandas as pd
import os
import tarfile
import glob

### ONLY RUN THIS BLOCK IF YOU NEED TO EXTRACT FILES AND MAKE THEM IN TO PARQUET

# Config
TAR_DIR = r"your-directory-here"   # folder with .tar files
OUTPUT_DIR = r"your-directory-here"
EXTRACT_DIR = r"your-directory-here"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)

# Charlotte bounding box
lat_min, lat_max = 34.714, 35.714
lon_min, lon_max = -81.443, -80.443

# Columns to keep
COLUMNS_TO_KEEP = ['icao24','callsign','time','lon','lat',
                   'baroaltitude','onground','velocity','heading','vertrate']


# Function to process one CSV file
def process_csv(file_path):
    try:
        filtered_chunks = []

        for chunk in pd.read_csv(file_path, usecols=COLUMNS_TO_KEEP, chunksize=500_000):
            mask = (chunk['latitude'].between(lat_min, lat_max)) & (chunk['longitude'].between(lon_min, lon_max))
            filtered_chunks.append(chunk[mask])

        if filtered_chunks:
            df_filtered = pd.concat(filtered_chunks)
            base_name = os.path.basename(file_path).replace(".csv", ".parquet")
            output_file = os.path.join(OUTPUT_DIR, base_name)
            df_filtered.to_parquet(output_file, engine='pyarrow', index=False)
            print(f"Saved {output_file} ({len(df_filtered)} flights)")
        else:
            print(f"No flights near Charlotte in {file_path}")
    except Exception as e:
        print(f"Failed {file_path}: {e}")



# Find all tar files in the directory
gz_files = glob.glob(os.path.join(EXTRACT_DIR, "*.csv.tar"))
print(f"Found {len(gz_files)} tar files.")
for file in gz_files:
    try:
        df = pd.read_csv(file, compression="tar")
        parquet_path = file.replace(".csv.tar", ".parquet")
        df.to_parquet(parquet_path, index=False)
        print(f"Converted: {os.path.basename(file)} → {os.path.basename(parquet_path)}")
    except Exception as e:
        print(f"Error converting {file}: {e}")

print("All conversions completed.")

# Save on memory by only using the relevant Charlotte flights
parquet_files = glob.glob(os.path.join(EXTRACT_DIR, "*.parquet"))
filtered_chunks = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_filtered = df[
        (df["lat"].between(lat_min, lat_max)) &
        (df["lon"].between(lon_min, lon_max))
    ]
    filtered_chunks.append(df_filtered)

# Combine only filtered parts
df_filtered_all = pd.concat(filtered_chunks, ignore_index=True)
print(f"Combined filtered DataFrame shape: {df_filtered_all.shape}")
df_filtered_all.to_csv("kclt_opensky_sept_2021_to_jun_2022.csv")



        


Here is the code I used to scrape the METAR database, I had to put which URL I wanted to scrape in here. They only allow up to a full month of codes to be pulled at once so I had to do multiple requests.

In [ ]:
# Scrape historical METAR data to match with flights
import requests
from bs4 import BeautifulSoup
import re
import sys

URL = ("")
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/120.0.0.0 Safari/537.36")
}

# Defining a function to grab the URL
def fetch(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=20)
    except Exception as e:
        print("ERROR: network/request failed:", e, file=sys.stderr)
        return None
    print("HTTP", resp.status_code)
    if resp.status_code != 200:
        print("Response length:", len(resp.text))
        # print a bit of the returned page for debugging
        print("Page snippet:\n", resp.text[:1000])
        return None
    return resp.text

# Extract the METARS from the data
def extract_metars_from_pre(text):
    soup = BeautifulSoup(text, "html.parser")
    # find <pre> blocks
    pres = soup.find_all("pre")
    metars = []
    for p in pres:
        block = p.get_text("\n").strip()
        for line in block.splitlines():
            line = line.strip()
            if line.upper().startswith(("METAR ", "SPECI ")):
                metars.append(line)
    return metars

def extract_metars_by_regex(text):
    # Capture any line that starts with METAR or SPECI
    pattern = re.compile(r'(?m)^\s*(?:METAR|SPECI)\b.*$')
    return [m.strip() for m in pattern.findall(text)]

# Put everything together and call the main function
def main():
    html = fetch(URL)
    if not html:
        print("No HTML returned; aborting.")
        return

    metars = extract_metars_from_pre(html)
    if not metars:
        # Search raw HTML with regex if it fails for whatever reason
        metars = extract_metars_by_regex(html)

    if not metars:
        print("No METAR/SPECI lines found. Dumping a short snippet to help debug:")
        print(html[:4000])
        return

    # Print count and first 20 lines
    print(f"Found {len(metars)} METAR/SPECI lines. Showing up to first 20:\n")
    for i, line in enumerate(metars[:20], start=1):
        print(f"{i:02d}: {line}")

    # Save all to a file for later processing
    out_file = "kclt_sep_06_2021.txt"
    with open(out_file, "w", encoding="utf-8") as f:
        f.write("\n".join(metars))
    print(f"\nSaved {len(metars)} lines to {out_file}")

if __name__ == "__main__":
    main()

In this section I converted all of the METAR codes to usable data and saved it.

In [ ]:
import re
from pathlib import Path
import pandas as pd
from metpy.io import parse_metar_to_dataframe

# Folder path
folder = Path(r"C:\Users\Rob\code_stuff\interview_pp\metar_data")  # Windows raw string

# Mapping from 3-letter month abbreviations to month numbers
MONTH_MAP = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6,
    "jul": 7, "aug": 8, "sep": 9, "oct": 10, "nov": 11, "dec": 12
}

all_dfs = []

# Grab the files
for path in folder.glob("*.txt"):
    fname = path.stem.lower() 
    
    # Extract month abbreviation from filename
    match = re.search(r'_(\w{3})_\d{1,2}_(\d{4})$', fname)
    if not match:
        print(f"Skipping {path.name} — name doesn't match pattern.")
        continue
    
    month_abbr, year_str = match.groups()
    month = MONTH_MAP.get(month_abbr)
    year = int(year_str)
    
    print(f"Parsing {path.name} → month={month}, year={year}")

    # Read file text
    text = path.read_text().strip()

    # Split into individual lines
    lines = [line.strip().rstrip('=') for line in text.splitlines() if line.strip()]
    
    for line in lines:
        try:
            df_block = parse_metar_to_dataframe(line, year=2022, month=month)
            df_block["source_file"] = path.name
            all_dfs.append(df_block)
        except Exception as e:
            print(f"Could not parse line in {path.name}: {line} → {e}")

# Combine all DataFrames
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"\n Parsed {len(df)} total METARs from {len(all_dfs)} lines.")
else:
    df = pd.DataFrame()
    print("No valid METAR files found.")

# Save to CSV
df.to_csv("metar_combined.csv", index=False)
print("Saved to metar_combined.csv")

In [ ]:
df.head()

## Flight data analysis and conversions

Converting from metric units here since that's not the standard for aviation in America 

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import contextily as ctx
from collections import Counter


# Read in the data and wrangle it a little
df = pd.read_csv("kclt_opensky_sept_2021_to_jun_2022.csv")

# Fill in some na values
df["squawk"] = df["squawk"].fillna("Missing")
df["callsign"] = df["callsign"].fillna("Missing")
df = df.dropna()

# Convert to datetime for ease
df["datetime"] = pd.to_datetime(df["time"], unit = "s")
df["hour"] = df["datetime"].dt.hour
df["minute"] = df["datetime"].dt.minute
df["day"] = df["datetime"].dt.day

# Group flights by icao & time to find the time difference
df.sort_values(["icao24", "datetime"])
df["time_diff"] = df.groupby("icao24")["datetime"].diff().dt.total_seconds().div(60)
df["time_diff"] = df["time_diff"].fillna(0)

# Convert all metrics to ft, kts
meters_to_feet = 3.28084
meters_per_s_to_knots = 1.94384

df["baroaltitude"] = df["baroaltitude"] * meters_to_feet
df["velocity"] = df["velocity"] * meters_per_s_to_knots
df["vertrate"] = df["vertrate"] * meters_to_feet

# Calculate the distance in feet from the airport
ft_per_dg_lat = 364173.24
airport_lat, airport_lon = 35.2140, -80.9431
dlat = df['lat'] - airport_lat
dlon = df['lon'] - airport_lon
df['dist_from_KCLT_ft'] = np.sqrt(
    (dlat * ft_per_dg_lat)**2 +
    (dlon * ft_per_dg_lat * np.cos(np.radians(airport_lat)))**2
)

# Calculate the bearing to the airport
lat1 = np.radians(airport_lat)
lon1 = np.radians(airport_lon)
lat2 = np.radians(df['lat'])
lon2 = np.radians(df['lon'])

dlon = lon2 - lon1

x = np.sin(dlon) * np.cos(lat2)
y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)

initial_bearing = np.degrees(np.arctan2(x, y))
df['bearing_to_airport'] = (initial_bearing + 360) % 360






In [ ]:
df.info()

Merging our METAR data in to the df

In [ ]:
# Merge the two dfs on datetime to get the weather data
df_metar = pd.read_csv("metar_combined.csv")
df_metar["date_time"] = pd.to_datetime(df_metar["date_time"])
df_metar = df_metar.sort_values("date_time")
df = df.sort_values("datetime")
df_merged = pd.merge_asof(
    df,
    df_metar,
    left_on='datetime',
    right_on='date_time',
    direction='nearest',      # nearest METAR before or after
    tolerance=pd.Timedelta('1h')  # max 1 hour difference
)



In [ ]:
# Clean up the data a little more with the added METAR data

cols_to_drop = [
 # Most of these had tons of NA vlaues
        "wind_gust", 
        "current_wx1", 
        "current_wx2", 
        "current_wx3", 
        "low_cloud_type", 
        "low_cloud_level", 
        "medium_cloud_type", 
        "medium_cloud_level",
        "high_cloud_type",
        "high_cloud_level",
        "highest_cloud_type",
        "highest_cloud_level",
        "eastward_wind",
        "northward_wind",
        "source_file",
        "Unnamed: 0",
        "latitude", # These are for METAR data
        "longitude",
]
df_merged = df_merged.drop(columns=cols_to_drop)
df_merged = df_merged.dropna()
df_merged.info()
df_merged.to_csv("kclt_opensky_and_metar_clean.csv")

## Creating Flight Lines

In order to visualize the airspace better I created a geopandas df of LineStrings grouped by time and icao24 with a threshold of 60 minutes.

In [ ]:
import geopandas as gpd
import pandas as pd
import contextily as ctx
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import numpy as np


df = pd.read_csv("kclt_opensky_and_metar_clean.csv")
# Convert to GeoDataFrame with line geometry per flight
# Save the ids and time for identifying arrivals and departures to add back to the normal df
flight_paths = []
ids = []
avg_vertrates = []
altitudes = []
start_time = []
end_time = []

# Define a time threshold of 1 hour
time_threshold = 60

for icao, icao_df in df.groupby("icao24"):
    icao_df = icao_df.sort_values("datetime").reset_index(drop=True)
    
    # Identify where the time gap exceeds an hour
    split_indices = [0]  # start of first segment
    for i, td in enumerate(icao_df["time_diff"]):
        if td > time_threshold:
            split_indices.append(i)
    split_indices.append(len(icao_df))  # end of last segment

    # Create LineStrings for each flight segment
    for start, end in zip(split_indices[:-1], split_indices[1:]):
        segment = icao_df.iloc[start:end]
        if len(segment) > 1:  # need at least 2 points for a LineString
            coords = list(zip(segment["lon"], segment["lat"], segment["baroaltitude"]))
            flight_paths.append(LineString(coords))
            ids.append(icao)
            avg_vertrates.append(segment.vertrate.mean())
            altitudes.append(segment.baroaltitude.mean()) 
            start_time.append(segment.datetime.iloc[0])
            end_time.append(segment.datetime.iloc[-1]) 

# Convert to gdf
gdf = gpd.GeoDataFrame({
    "icao24": ids,
    "avg_vertrate": avg_vertrates,
    "avg_altitude": altitudes,
    "start_time" : start_time,
    "end_time" : end_time
    }, geometry=flight_paths, crs="EPSG:4326")

# Convert to Web Mercator for basemap
gdf_web = gdf.to_crs(epsg=3857)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
gdf_web.plot(ax=ax, color="Cyan", alpha=0.07, linewidth=0.06)

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatterNoLabels)

ax.set_axis_off()
plt.savefig("flight_paths.png", dpi=500, bbox_inches="tight", transparent=True)
plt.show()


## Attempt to define Departures, Arrivals andd Cruising flights

Here I had to define a cutoff for what we considered cruising. Ideally this data would be provided or derived from arrival/destination information but that was not provided in our dataset.

In [ ]:

# Define the start and end points
def get_start_alt(geom):
    return geom.coords[0][2] if len(geom.coords[0]) > 2 else None

def get_end_alt(geom):
    return geom.coords[-1][2] if len(geom.coords[-1]) > 2 else None

gdf["alt_start"] = gdf.geometry.apply(get_start_alt)
gdf["alt_end"] = gdf.geometry.apply(get_end_alt)
gdf = gdf.dropna()


# Determine arrivals and departures
def classify_by_alt(row):
    z_start = row["alt_start"]
    z_end = row["alt_end"]
    z_rate = row["avg_vertrate"]
    # Handle missing altitudes
    if z_start is None or z_end is None:
        return "unknown"
    
    

    # Cruising: stays above 15000 ft
    if (z_start >= 15000) and (z_end >= 15000) or (abs(z_rate)<5):
        return "Cruising"
    # Arrival has a rapid descent
    elif (z_rate <= -5):
        return "Arrival"
    # Departure has a rapid ascent
    elif(z_rate >= 5):
        return "Departure"
    else:
        return "other"

# Avg vertrate within 500ft/min
gdf["flight_phase"] = gdf.apply(classify_by_alt, axis=1)
gdf.groupby("flight_phase").size()


In [ ]:
# Save
gdf.to_csv("flight_paths.csv")

In [ ]:
# Convert to datetime and determine start times
df["datetime"] = pd.to_datetime(df["datetime"])
gdf["start_time"] = pd.to_datetime(gdf["start_time"])
gdf["end_time"] = pd.to_datetime(gdf["end_time"])

# Build flights table
flights = gdf[["icao24", "start_time", "end_time", "flight_phase"]].copy()

# Sort both by icao24 and time
df = df.sort_values("datetime").reset_index(drop=True)
flights = flights.sort_values("start_time").reset_index(drop=True)

#Had to vectorize because this took forever
merged = pd.merge_asof(
    df,
    flights,
    by="icao24",
    left_on="datetime",
    right_on="start_time",
    direction="backward",
    allow_exact_matches=True
)

mask = merged["datetime"] <= merged["end_time"]
merged.loc[~mask, "flight_phase"] = None  # points outside the segment get NaN


# print(filtered)

df = merged

# Save new df
df.head()


In [ ]:
# Save all of this for our later models
cols_to_drop = [  
        "station_id",
        "date_time",
        "remarks",
        "start_time",
        "end_time",
        "squawk",
        "callsign"
]
df_save = df.drop(columns=cols_to_drop)
df_save = df_save.dropna()
df_save.info()
df_save.to_csv("model_data.csv")


I wanted to map by altitude here

In [ ]:
# Create bins
bin_size = 100
gdf["altitude_bin"] = (gdf["avg_altitude"] // bin_size) * bin_size

# Convert to Web Mercator for basemap
gdf_web = gdf.to_crs(epsg=3857)

# Plot
fig, ax = plt.subplots(figsize=(10, 10), facecolor="black")

gdf_web.plot(
    ax=ax,
    column="altitude_bin",
    cmap="RdYlBu",
    linewidth=0.08,
    alpha=0.08
)

# Add dark basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatterNoLabels)

# Create colorbar with discrete ticks for each bin
unique_bins = sorted(gdf_web["altitude_bin"].unique())
sm = plt.cm.ScalarMappable(
    cmap="RdYlBu", 
    norm=plt.Normalize(vmin=min(unique_bins), vmax=max(unique_bins))
)
sm._A = []
tick_locs = [b for b in unique_bins if b % 4000 == 0]
cbar = plt.colorbar(sm, ax=ax, ticks=tick_locs, shrink=.8)
cbar.set_label("Altitude (ft)", color="black", fontsize=20)
cbar.ax.yaxis.set_tick_params(color='black')
cbar.ax.yaxis.set_tick_params(labelcolor='black')
cbar.ax.tick_params(labelsize=14)

# Configure plot
ax.set_axis_off()
plt.tight_layout()
plt.savefig("flight_paths_by_alt.png", dpi=500, bbox_inches="tight", transparent=True)
plt.show()

Here we can change what we're plotting. Right now it's class B vs class A airspace but it can also produce the arrivals and departures graph

In [ ]:
from matplotlib.lines import Line2D
# Define colors
phase_colors = {
    "Cruising": "#FF13F0",       # unique color
    "Arrivals & Departures": "cyan"  # combined color for arrivals/departures
}

# Filter data for plotting
gdf_plotting = gdf.to_crs(epsg=3857)
gdf_filtered = gdf_plotting[gdf_plotting["flight_phase"].isin(["Arrival", "Departure", "Cruising"])]

# Map color: arrivals/departures get same color, cruising stays separate
def map_phase_color(phase):
    if phase in ["Arrival", "Departure"]:
        return phase_colors["Arrivals & Departures"]
    else:
        return phase_colors["Cruising"]

colors = gdf_filtered["flight_phase"].map(map_phase_color)

# Plot flights
ax = gdf_filtered.plot(
    figsize=(12, 12),
    linewidth=0.08,
    alpha=0.08,
    color=colors
)

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatterNoLabels)

# Create legend with custom labels
handles = [
    Line2D([0], [0], color=phase_colors["Arrivals & Departures"], lw=2, label="Class B Airspace"),
    Line2D([0], [0], color=phase_colors["Cruising"], lw=2, label="Class A Airspace")
]

ax.legend(handles=handles, loc='upper right', fontsize=20)
ax.set_axis_off()

plt.savefig("flight_paths_by_phase.png", dpi=500, bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
gdf.head()


## Analyzing the Data with Densities

Getting an feel for the dataset with density graphs

In [ ]:
import seaborn as sns

# Define a function to easily plot a bunch at once for the PP
def plot_dens(var, xlabel, xlimit):
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(
        data=df,
        x=var,      
        fill=True,
        color="blue",
        alpha=0.6,
        linewidth=1.5
    )

    ticks = plt.yticks()[0]  # get default tick locations
    plt.yticks(ticks, [f"{t*100:.1f}%" for t in ticks]) 
    plt.xlabel(xlabel, fontsize = 20)
    plt.ylabel("Density",fontsize = 20)
    plt.xlim(xlimit[0], xlimit[1])
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.grid(True, alpha=0.3)

plot_dens("baroaltitude", "Altitude (feet)", (0,60000))
plt.axvline(x=10000, color="red", linestyle="--", linewidth=2, label="x = 250", alpha = .5)
plt.savefig("altitude_density.png", dpi=500, bbox_inches="tight")
plt.show()






In [ ]:
plot_dens("velocity", "Velocity (knots)", (0,600))
plt.axvline(x=250, color="red", linestyle="--", linewidth=2, label="x = 250", alpha = .5)
plt.savefig("velocity_density.png", dpi=500, bbox_inches="tight")
plt.show()

In [ ]:
plot_dens("vertrate", "Vertical Climbing Rate (ft/s)", (-80, 80))
plt.savefig("vertrate_density.png", dpi=500, bbox_inches="tight")
plt.show()

In [ ]:
plot_dens("bearing_to_airport", "Bearing (Degrees from Charlotte)",(0,360))
plt.show()

In [ ]:
# df = pd.read_csv("kclt_opensky_and_metar_clean.csv")

def plot_metar(data, xlabel):

    plt.figure(figsize = (10,6))
    plt.hist(data, bins=10, color="blue", alpha=.5, edgecolor="black")
    plt.xlabel(xlabel, fontsize=24)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.ylabel('Count', fontsize =24)
    plt.savefig("wind_dir.png", dpi=500, bbox_inches="tight")
    plt.show()

# plot_metar(df["air_temperature"], "Temperature (Degrees Celsius)")
# plot_metar(df["dew_point_temperature"], "Dew Point (Degrees Celsius)")
# plot_metar(df["wind_speed"],  "Wind Speed (Knots)")
plot_metar(df["wind_direction"], "Wind Direction (Degrees from true North)")




In [ ]:
df.info()

In [ ]:
# Hexbin plot of velocity and altitude
x = df['velocity']
y = df['baroaltitude']

plt.figure(figsize=(10, 6))

hb = plt.hexbin(
    x, y,
    gridsize=500,       # number of hexagons along x-axis
    cmap='plasma',
    mincnt=1,
    bins = "log"        
)
plt.colorbar(hb).set_label("Count (log scale)", size = 14)
plt.xlabel('Velocity (knots)', fontsize = 20)
plt.ylabel('Barometric Altitude (ft)', fontsize = 20)
plt.ylim(0,50000)
plt.xlim(0,600)
plt.tick_params(labelsize=14)
plt.hlines(10000,0,600, linestyles = '--', colors = 'red', alpha = .8)

plt.savefig("velocity_baro.png", dpi=500, bbox_inches="tight")
plt.show()


## Bayesian Network

Start of the Bayesian network, I had to filter a ton of columns due to skew.

In [ ]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import K2, HillClimbSearch, BayesianEstimator
from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer

# Load data
bn_data = pd.read_csv("model_data.csv")
flight_df = pd.read_csv("flight_paths.csv")
bn_df = bn_data.drop(columns=[
                            # Remove unwanted columns
                            'Unnamed: 0.1',
                            'Unnamed: 0',
                            'onground', 
                            'alert', 
                            'spi', 
                            'elevation', 
                            'visibility',  
                            'current_wx2_symbol', 
                            'current_wx3_symbol',
                            'cloud_coverage',
                            'dist_from_KCLT_ft',
                            "icao24",
                            "datetime",
                            "lastcontact",
                            "lastposupdate",
                            'day',
                            'minute',
                            'hour',
                            'altimeter',
                            'air_pressure_at_sea_level',
                            'geoaltitude',
                            'time',
                            'air_temperature',
                            'time_diff'                    
])




In [ ]:
bn_df.info()

Changing back to a discrete label for weather

In [ ]:
# Rename the weather symbols back to their discrete state
bn_df["current_wx1_symbol"].value_counts()
wx_map = {
    0: "Clear",
    17: "Fog",
    61: "Rain",
    95: "Thunderstorm",
    97: "Thunderstorm"
}

bn_df["wx_label"] = bn_df["current_wx1_symbol"].map(lambda x: wx_map.get(int(x % 100), "Unknown"))
bn_df = bn_df.drop(columns=['current_wx1_symbol'])



Finding skewed columns

In [ ]:
from scipy.stats import skew
import pandas as pd

# Compute skewness for all numeric columns
skewness = bn_df.select_dtypes(include=['number']).apply(skew, nan_policy='omit')

# Sort by absolute skewness
skewness = skewness.sort_values(key=abs, ascending=False)

skew_limit = 1.0  # can adjust for stricter filter
skewed_cols = skewness[abs(skewness) > skew_limit].index.tolist()

print("Highly skewed columns:")
print(skewed_cols)

Discretizing our columns

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer, PowerTransformer
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination

# Automatically select numeric columns except 'flight_phase'
cols_to_discretize = bn_df.select_dtypes(include=['float64', 'int64']).columns.difference(['flight_phase','wx_label'])

# Remove near-constant columns to avoid KBins warning
skewed_cols = [col for col in skewed_cols if bn_df[col].nunique() > 1]

# Initialize PowerTransformer
pt = PowerTransformer(method='yeo-johnson', standardize=True)  # Yeo-Johnson works with negative values too

# Fit and transform
bn_df[skewed_cols] = pt.fit_transform(bn_df[skewed_cols])

# Initialize KBinsDiscretizer
kb = KBinsDiscretizer(n_bins=9, encode="ordinal", strategy="quantile")

# Fit on the data
kb.fit(bn_df[cols_to_discretize])

# Save bin edges for mapping real values to bins later
bin_edges = {col: kb.bin_edges_[i] for i, col in enumerate(cols_to_discretize)}

# Transform the data
bn_df[cols_to_discretize] = kb.transform(bn_df[cols_to_discretize])


Making the Bayesian network. I had to define the initial structure. GES would sometimes ignore the required edges and forbidden edges but temporal fixed a lot of my issues.

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BIC, GES, ExpertKnowledge

expert_knowledge = ExpertKnowledge(
    # Edges that must appear in the final DAG
    required_edges=[('lat', 'bearing_to_airport'),
                    ('lon', 'bearing_to_airport'),
                    ('flight_phase', 'velocity'),
                    ('baroaltitude', 'velocity'),
                    ('vertrate','baroaltitude'),
                    ('flight_phase', 'baroaltitude'),
                    ('flight_phase', 'vertrate')
                    
    ],
    
    # Edges that must not appear
    forbidden_edges=[

    # Heading effects
    ('heading', 'baroaltitude'),
    ('heading', 'vertrate'),
    ('bearing_to_airport','dew_point_temperature'), 

    # Baroaltitude effects
    ('baroaltitude', 'flight_phase'),
    ('baroaltitude', 'velocity'),

    # Wind effects
    ('wind_direction', 'lat'),
    ('wind_direction', 'lon'),
    ('wind_speed', 'lat'),
    ('wind_speed', 'lon'),

    # Dew point / weather effects
    ('dew_point_temperature', 'lon'),
    ('dew_point_temperature', 'lat')
    ],
    
    # Order that edges must appear in
    temporal_order=[
    ('dew_point_temperature', 'wind_direction', 'wind_speed', 'wx_label'),  # weather first
    ('lat', 'lon', 'bearing_to_airport'),  # position
    ('flight_phase', 'baroaltitude', 'vertrate', 'velocity'),  # flight dynamics
    ('heading',)  # heading last
]
)

hc = GES(bn_df)
dag = hc.estimate(scoring_method=BIC(bn_df), expert_knowledge=expert_knowledge)

bn_model = DiscreteBayesianNetwork(dag.edges())
bn_model.fit(bn_df, estimator=MaximumLikelihoodEstimator)


In [ ]:
# Save the BN
import pickle

with open("bn_model.pkl", "wb") as f:
    pickle.dump(bn_model, f)

In [ ]:
bn_model.nodes

Testing a flight phase here for verification.

In [ ]:
# Define a function that can map the real values of a flight to their corresponding bins
import warnings
infer = VariableElimination(bn_model)


def map_to_bin(col, value):
    warnings.filterwarnings("ignore", category=UserWarning)
    if col in cols_to_discretize:
        # Transform value only if column was included in PowerTransformer
        if col in skewed_cols:
            # Create placeholder for all skewed features (same length as fit)
            tmp = np.zeros((1, len(skewed_cols)))
            col_idx_skewed = list(skewed_cols).index(col)
            tmp[0, col_idx_skewed] = value

            # Transform and extract the corresponding transformed value
            val_t = pt.transform(tmp)[0, col_idx_skewed]
        else:
            val_t = value  # leave untouched if not skewed
        
        # Use KBinsDiscretizer bin edges for this column
        col_idx = list(cols_to_discretize).index(col)
        edges = kb.bin_edges_[col_idx]
        bin_idx = np.digitize(val_t, edges[1:-1])
        return np.clip(bin_idx, 0, len(edges)-2)
    else:
        return value  # categorical or excluded variables


# Define evidence using raw values
evidence = {
    'velocity': map_to_bin('velocity', 450),
    'vertrate': map_to_bin('vertrate', 0),
    'baroaltitude': map_to_bin('baroaltitude', 35000),
    
}

query_result = infer.query(variables=["flight_phase"], evidence=evidence)
print(query_result)


Here I'm going to analyze what our Bayesian network finds

In [ ]:
# Load data
import warnings
import pandas as pd
bn_data = pd.read_csv("model_data.csv")
bn_data = bn_data.drop(columns=[
                            # Remove unwanted columns
                            'Unnamed: 0.1',
                            'Unnamed: 0',
                            'onground', 
                            'alert', 
                            'spi', 
                            'elevation', 
                            'visibility',  
                            'current_wx2_symbol', 
                            'current_wx3_symbol',
                            'cloud_coverage',
                            'dist_from_KCLT_ft',
                            "icao24",
                            "datetime",
                            "lastcontact",
                            "lastposupdate",
                            'day',
                            'minute',
                            'hour',
                            'altimeter',
                            'air_pressure_at_sea_level',
                            'geoaltitude',
                            'time',
                            'air_temperature',
                            'time_diff'  
]
)

def map_to_bin(col, value):
    warnings.filterwarnings("ignore", category=UserWarning)
    if col in cols_to_discretize:
        # Transform value only if column was included in PowerTransformer
        if col in skewed_cols:
            # Create placeholder for all skewed features (same length as fit)
            tmp = np.zeros((1, len(skewed_cols)))
            col_idx_skewed = list(skewed_cols).index(col)
            tmp[0, col_idx_skewed] = value

            # Transform and extract the corresponding transformed value
            val_t = pt.transform(tmp)[0, col_idx_skewed]
        else:
            val_t = value  # leave untouched if not skewed
        
        # Use KBinsDiscretizer bin edges for this column
        col_idx = list(cols_to_discretize).index(col)
        edges = kb.bin_edges_[col_idx]
        bin_idx = np.digitize(val_t, edges[1:-1])
        return np.clip(bin_idx, 0, len(edges)-2)
    else:
        return value  # categorical or excluded variables


In [ ]:
import pickle

# Load your model
with open("bn_model.pkl", "rb") as f:
    bn_model = pickle.load(f)

# model.edges() gives parent → child relationships
edges = list(bn_model.edges())
print("Dependencies (Parent -> Child):")
for parent, child in edges:
    print(f"{parent} -> {child}")

In [ ]:
from scipy.stats import entropy

bn_data["datetime"] = pd.to_datetime(bn_data["datetime"])



# Filter flight
icao24 = '06a052'
start_time = "2022-06-13 00:16:20"
end_time = "2022-06-13 00:20:20"

flight_df = bn_data[
    (bn_data['icao24'] == icao24) &
    (bn_data["datetime"] >= start_time) &
    (bn_data["datetime"] <= end_time)
].sort_values("datetime").reset_index(drop=True)


# Take last row, modify, and append
last_row = flight_df.iloc[-1].copy()
last_row["velocity"] = last_row["velocity"] - 500
last_row["datetime"] = last_row["datetime"] + pd.Timedelta(seconds=10)

flight_df = pd.concat([flight_df, last_row.to_frame().T], ignore_index=True)

flight_df = flight_df.drop(columns=[
                            # Remove unwanted columns
                            'Unnamed: 0.1',
                            'Unnamed: 0','onground', 
                            'alert', 'spi', 'elevation', 
                            'visibility', 'current_wx1_symbol', 
                            'current_wx2_symbol', 
                            'current_wx3_symbol',
                            'cloud_coverage',
                            'dist_from_KCLT_ft'
                            
])
                        
flight_df["datetime"] = pd.to_datetime(flight_df["datetime"])
flight_df.tail()


I wanted to test the Bayesian net with a synthetic scenario to see how it picks up on anomalies

In [ ]:
# Create a fake flight to demonstrate the rapid change in out anomaly score
import pandas as pd

n_maintain = 60      # steps maintaining altitude
n_fall = 60           # steps in dive
alt_start = 35000
v_start = 500     # starting altitude in ft
dv = 10
dt = 1               # time step in seconds
vertrate = -400



# Velocity
dummy_flight_1 = pd.DataFrame(columns = bn_df.columns)   
dummy_flight_1["velocity"] = [500] *(n_maintain + n_fall) 

# Vertical rate - flight just dive bombs
dummy_flight_1["vertrate"] = ([0] * n_maintain) + ([vertrate] * n_fall)


# Altitude
alt_end = []
for i in range(n_fall):
    alt_end.append(alt_start + (vertrate * i))
dummy_flight_1["baroaltitude"] = [alt_start] * (n_maintain) + alt_end
dummy_flight_1['flight_phase'] = 'Cruising'
dummy_flight_1['wx_label'] = 'Clear'
dummy_flight_1['bearing_to_airport'] = 340


Defining a compute anomaly score function

In [ ]:
from pgmpy.inference import VariableElimination


# Make a function that can compute the joint probability
evidence_cols = ['baroaltitude', 'vertrate', 'velocity', 'flight_phase']

epsilon = 1e-12  # to avoid log(0)

# Initialize inference object
infer = VariableElimination(bn_model)

def compute_anomaly_score(row):
    joint_prob = 1.0

    for var in evidence_cols:
        val = row[var]

        # Map to bin
        bin_value = map_to_bin(var, val)

        # Build evidence excluding current variable
        other_evidence = {v: map_to_bin(v, row[v]) for v in evidence_cols if v != var}

        # Query probability of current variable given other evidence
        res = infer.query(variables=[var], evidence=other_evidence)

        # Find the index of the bin value in the CPD states
        try:
            state_idx = res.state_names[var].index(bin_value)
        except ValueError:
            # If somehow the value is outside known bins, assign a very small probability
            joint_prob *= epsilon
            continue

        joint_prob *= res.values[state_idx]

    return -np.log(joint_prob + epsilon)
# Apply to each row
dummy_flight_1['anomaly_score'] = dummy_flight_1.apply(compute_anomaly_score, axis=1)

In [ ]:
# 50k row sample for estimate on density
bn_sample = bn_data.sample(50000)

bn_sample['anomaly_score'] = bn_sample.apply(compute_anomaly_score, axis=1)

In [ ]:
change = dummy_flight_1.iloc[60]['anomaly_score']
orig = dummy_flight_1.iloc[0]['anomaly_score']
final = dummy_flight_1.iloc[-1]['anomaly_score']

first_jump = change - orig
second_jump = final - orig
std = second_jump/2.45

print(first_jump)
print(second_jump)
print(std)

Flight path visualiztation

In [ ]:
import matplotlib.pyplot as plt

# Plot the flight path
time  = range(120)
plt.figure(figsize=(10, 6))
plt.plot(time, dummy_flight_1["baroaltitude"], label = "Flight Path")
plt.ylabel("Barometric Altitude", fontsize = 20)
plt.xlabel("Time in Seconds", fontsize = 20)
plt.tick_params(labelsize = 14)
plt.legend()
plt.savefig("dummy_path.png", dpi=500)
plt.show()


Anomaly score visualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the anomaly score
plt.figure(figsize=(10, 6))
plt.plot(dummy_flight_1['anomaly_score'])
plt.ylabel('Anomaly Score', fontsize = 20)
plt.xlabel('Time (seconds)', fontsize = 20)
plt.grid(True, alpha = .3)
plt.vlines(60, 0, 30, linestyles='--', colors = 'red', alpha = .3)
plt.ylim(0,28)

plt.savefig("anomaly.png", dpi=500)
plt.show()

In [ ]:
print(bn_sample['anomaly_score'].quantile(.95))
print(bn_sample['anomaly_score'].std())

Density of normal samples visualization

In [ ]:
# Plot the denisty of anomaly scores across normal conditions
import seaborn as sns
def plot_score(var, xlabel, xlimit):
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(
        data=bn_sample,
        x=var,      
        fill=True,
        color="blue",
        alpha=0.6,
        linewidth=1.5
    )
    ticks = plt.yticks()[0]
    plt.yticks(ticks, [f"{t*100:.1f}%" for t in ticks]) 
    plt.xlabel(xlabel, fontsize = 20)
    plt.ylabel("Density",fontsize = 20)
    plt.xlim(xlimit[0], xlimit[1])
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.grid(True, alpha=0.3)

plot_score("anomaly_score", "Anomaly Score",(0,30))
plt.savefig("anomaly_score_density.png", dpi=500, bbox_inches="tight")
plt.show()

## Autoencoder

In [ ]:
import pandas as pd
df = pd.read_csv("model_data.csv")
df.info()

In [ ]:
# tanh is the way to go
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow import keras
import tensorflow as tf

# Get rid of unwanted columns
cols_to_drop = [
                'icao24',  
                'alert', 
                'spi', 
                'datetime',
                'Unnamed: 0',
                'lastposupdate',
                'lastcontact',
                'hour',
                'minute',
                'day',
                'current_wx1_symbol',
                'current_wx2_symbol',
                'current_wx3_symbol',
                'geoaltitude',
                'air_temperature',
                'altimeter',
                'elevation',
                'visibility',
                'cloud_coverage',
                'onground',
                'time_diff',
                'dist_from_KCLT_ft',
                'air_pressure_at_sea_level',
                'time',
                'flight_phase',
                'Unnamed: 0.1'




]

# Scale X
X = df.drop(columns=cols_to_drop)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
features = X.columns

X.info()


Buildin a function to run multiple configs in succession and save their history, this was useful for running when I was away from my computer.

In [ ]:
import json

# Build an autoencoder function to run multiple configs
def build_autoencoder(input_dim, hidden_units1, hidden_units2, encoding_dim, l1=1e-4):
    # Encoder
    input_layer = keras.Input(shape=(input_dim,))
    x = keras.layers.Dense(hidden_units1, activation='tanh')(input_layer)
    x = keras.layers.Dense(hidden_units2, activation='tanh')(x)
    encoded = keras.layers.Dense(encoding_dim, activation='tanh',
                                 activity_regularizer=keras.regularizers.L1(l1))(x)
    
    # Decoder
    x = keras.layers.Dense(hidden_units2, activation='tanh')(encoded)
    x = keras.layers.Dense(hidden_units1, activation='tanh')(x)
    decoded = keras.layers.Dense(input_dim, activation='linear')(x)

    autoencoder = keras.Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder


input_dim = X_scaled.shape[1]

configs = [
        (10, 8, 6, "45"),
       

]


# save history for later
for hidden_units1, hidden_units2, encoding_dim, name in configs:
    autoencoder = build_autoencoder(input_dim, hidden_units1, hidden_units2, encoding_dim)
    history = autoencoder.fit(
        X_scaled, X_scaled,
        epochs=10,
        batch_size=64,
        validation_split=0.3,
        verbose=1
    )
    autoencoder.save(f"autoencoder_{name}.keras")
    with open(f"autoencoder_{name}_history.json", "w") as f:
        json.dump(history.history, f)

Plotting the validation loss vs training loss

In [ ]:
import matplotlib.pyplot as plt
hist = pd.read_json("autoencoder_45_history.json")
import matplotlib.pyplot as plt

# Get loss values
train_loss = hist["loss"]
val_loss = hist["val_loss"]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(train_loss, label="Training Loss", linewidth=2)
plt.plot(val_loss, label="Validation Loss", linewidth=2)

plt.ylabel("Loss", fontsize=20)
plt.xlabel("Epochs", fontsize=20)
plt.legend(fontsize=14)
plt.tick_params(labelsize=14)
plt.grid(True, alpha = .3)
plt.show()

I had to get the layer we were actually using here, since I ran different configs they all had different names. I passed a null vector through my autoencodedr to get the value. Latent space is the smallest layer.

In [ ]:
from tensorflow.keras.models import load_model, Model
import numpy as np

# I didn't know how to save the latent layer name so this helped
autoencoder = load_model("autoencoder_45.keras")

# Dummy input to get the layer name
dummy_input = np.zeros((1, 10))

# Run a forward pass to build layer shapes
_ = autoencoder.predict(dummy_input)

# Check shapes
for layer in autoencoder.layers:
    try:
        intermediate_model = Model(inputs=autoencoder.input, outputs=layer.output)
        output = intermediate_model.predict(dummy_input)
        print(layer.name, output.shape)
    except Exception as e:
        print(layer.name, "Error:", e)





In [ ]:
# Load the encoder
encoder = Model(
    inputs=autoencoder.input,
    outputs=autoencoder.get_layer("dense_8").output
)

Making our mean vector and covariance matrix for Mahalanobis distance.

In [ ]:
# Predict the Latent vector
latent_X = encoder.predict(X_scaled)

# Mean vector
mean_vector = np.mean(latent_X, axis=0)

# Covariance matrix
cov_matrix = np.cov(latent_X, rowvar=False)

# Inverse covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

scipy has mahalanobis distance in it's package

In [ ]:
from scipy.spatial.distance import mahalanobis

# Compute Mahalanobis Distances
m_distances = np.array([
    mahalanobis(row, mean_vector, inv_cov_matrix)
    for row in latent_X
])

# Add distances to X_scaled DataFrame if you want
X_df = pd.DataFrame(X_scaled, columns=X.columns)  # replace feature_names with your column names
X_df['mahalanobis_distance'] = m_distances

In [ ]:
# Drop some columns to read easily
cols_to_drop = [
                 
                'alert', 
                'spi', 
                'datetime',
                'Unnamed: 0.1',
                'Unnamed: 0',
                'lastposupdate',
                'lastcontact',
                'hour',
                'minute',
                'day',
                'current_wx1_symbol',
                'current_wx2_symbol',
                'current_wx3_symbol',
                'geoaltitude',
                'air_temperature',
                'altimeter',
                'elevation',
                'visibility',
                'cloud_coverage',
                'onground',
                'time_diff',
                'dist_from_KCLT_ft',
                'air_pressure_at_sea_level',
                'flight_phase'
]
# df = df.drop(columns = cols_to_drop)
pd.set_option("display.max_rows", 100)
df.nlargest(100,'mahalanobis_distance').loc[df['velocity'] > 400].count()

Here I found some true anomalies

In [ ]:
# Group by time again and take the values only an hour away
df = df.sort_values(["icao24", "time"])
df["time_diff"] = df.groupby("icao24")["time"].diff().fillna(0)
df["flight_id"] = (df["time_diff"] > 3600).cumsum() 

# Get the scores for flights in the 99th percentils
flight_scores = (
    df.groupby(["icao24", "flight_id"])["mahalanobis_distance"]
      .agg(["mean", "max", lambda x: np.percentile(x, 99)])
      .rename(columns={"<lambda_0>": "p99"})
      .reset_index()
)
flight_scores.rename(columns={"<lambda_0>": "p99"}, inplace=True)
flight_scores = flight_scores.reset_index()
flight_scores['icao24'].value_counts()
cutoff_99 = np.percentile(df["mahalanobis_distance"], 99)
print(cutoff_99)


In [ ]:
# Identify some weird flights
df['time'] = pd.to_datetime(df['time'], unit = 's')
df.loc[(df['mahalanobis_distance'] >= 2) & (df['vertrate'] >= 15) & (df['icao24'] == 'a6f6dd')].tail(200)

In [ ]:
start = '2022-03-28 03:26:20'
end = '2022-03-28 03:30:50'

# Mean flight Mahalanobis Distance
df_timeframe = df[(df['time'] >= start) & (df['time'] <= end) & (df['icao24'] == 'a6f6dd')]
print(df_timeframe['mahalanobis_distance'].mean())



Plotting out abnormal flight path

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import LineString

# Convert df_timeframe to GeoDataFrame
flight_gdf = gpd.GeoDataFrame(
    df_timeframe,
    geometry=gpd.points_from_xy(df_timeframe["lon"], df_timeframe["lat"]),
    crs="EPSG:4326"
).sort_values("time")

# Create a LineString for the flight
flight_line = LineString(flight_gdf.geometry.tolist())
flight_line_gdf = gpd.GeoDataFrame(
    {"icao24": [df_timeframe["icao24"].iloc[0]]},
    geometry=[flight_line],
    crs="EPSG:4326"
)

# Convert both gdf_plotting and flight_line_gdf to Web Mercator
gdf_plotting = gdf.to_crs(epsg=3857)
flight_line_gdf = flight_line_gdf.to_crs(epsg=3857)

# Filter arrivals and departures
mask = gdf_plotting["flight_phase"].isin(["Departure"])
gdf_filtered = gdf_plotting[mask]
colors = gdf_filtered["flight_phase"].map({
    
    "Departure": "cyan"
})

# Plot
ax = gdf_filtered.plot(
    figsize=(12, 12),
    linewidth=0.08,
    alpha=0.15,
    color=colors
)

# Overlay your single flight path in bright red
flight_line_gdf.plot(ax=ax, color="red", linewidth=2, label="Selected Flight")

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatterNoLabels)

# Add legend
handles = [plt.Line2D([0], [0], color=color, lw=2, label=phase) 
           for phase, color in {"Departure": "cyan"}.items()]
handles.append(plt.Line2D([0], [0], color="red", lw=2, label="Selected Flight"))
ax.legend(handles=handles, loc='upper right', fontsize=20)

ax.set_axis_off()
plt.savefig("flight_paths_by_phase_with_selected.png", dpi=500, bbox_inches="tight", transparent=True)
plt.show()

Extra analysis of variables

In [ ]:
# Playing aroundd with some plots to determine why a flight was anomalous
def plot_flight(x,y):

    plt.figure(figsize = (10,6))
    plt.plot(x,y)
    plt.xticks([])
    plt.yticks()
    plt.show()

plot_flight(df_timeframe['time'],df_timeframe['vertrate'])
plot_flight(df_timeframe['lat'],df_timeframe['lon'])
plot_flight(df_timeframe['time'],df_timeframe['velocity'])
plot_flight(df_timeframe['time'], df_timeframe['bearing_to_airport'])
plot_flight(df_timeframe['time'], df_timeframe['baroaltitude'])


In [ ]:
# Density plot of Mahalanobis Distance
def plot_dens(var, xlabel, xlimit):
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(
        data=X_df,
        x=var,      
        fill=True,
        color="blue",
        alpha=0.6,
        linewidth=1.5   
    )
    ticks = plt.yticks()[0]
    plt.yticks(ticks, [f"{t*100:.1f}%" for t in ticks]) 
    plt.xlabel(xlabel, fontsize = 20)
    plt.ylabel("Density",fontsize = 20)
    plt.xlim(xlimit[0], xlimit[1])
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.grid(True, alpha=0.3)

plt.savefig("mahal_density.png", dpi=500, bbox_inches="tight")
plot_dens("mahalanobis_distance", "Mahalanobis Distance",(0,8))
plt.show()

## Extra
I was playing around with another fake flight, decided to go with a real one

In [ ]:
import pandas as pd

n_maintain = 60      # steps maintaining altitude
n_fall = 60          # steps in dive
alt_start = 20000
v_start = 500
dv = 10
dt = 1
vertrate = -400

icao_id = 'c06a7b'
df_icao = df.loc[df['icao24'] == icao_id]
df_icao = df_icao[:6]

dummy_flight_2 = pd.DataFrame(columns=X.columns)

# Only apply linspace to numeric columns
for col in df_icao.columns:
    if pd.api.types.is_numeric_dtype(df_icao[col]):
        dummy_flight_2[col] = np.linspace(df_icao[col].min(), df_icao[col].max(), 120)
    else:
        # For non-numeric columns, just repeat the first value or fill with None
        dummy_flight_2[col] = [df_icao[col].iloc[0]] * 120 if len(df_icao[col]) > 0 else [None] * 120

# v_end = []
# for i in range(n_fall):
#     v_end.append(v_start + dv)    
dummy_flight_2["velocity"] = [500] *(n_maintain + n_fall) 

# Vertical rate
dummy_flight_2["vertrate"] = ([0] * n_maintain) + ([vertrate] * n_fall)


# Altitude
alt_end = []
for i in range(n_fall):
    alt_end.append(alt_start + (vertrate * i))
dummy_flight_2["baroaltitude"] = [alt_start] * (n_maintain) + alt_end
dummy_flight_2['flight_phase_Arrival'] = False
dummy_flight_2['flight_phase_Departure'] = False
dummy_flight_2['flight_phase_Cruising'] = True
dummy_flight_2 = dummy_flight_2.iloc[:, :10]
dummy_flight_2.info()


In [ ]:
from scipy.spatial.distance import mahalanobis

# Assume dummy_flight_2 is your new flight dataframe with same 10 features
df_flight_scaled = scaler.transform(dummy_flight_2)  # same scaler used for X_scaled
latent_flight = encoder.predict(df_flight_scaled)
# Vectorized computation
diff = latent_flight - mean_vector
m_distances = np.array([
    mahalanobis(x, mean_vector, inv_cov_matrix) 
    for x in latent_flight
])

# Add to dataframe
dummy_flight_2['mahalanobis_distance'] = m_distances
dummy_flight_2['mahalanobis_distance'].plot()

In [ ]:
plt.plot(dummy_flight_2['mahalanobis_distance'])
plt.ylabel("Mahalanobis Distance", fontsize=20)
plt.xlabel("Time (seconds)", fontsize=20)
plt.tick_params(labelsize=14)
plt.grid(True, alpha = .3)
plt.show()

In [ ]:
print(dummy_flight_2[X.columns].shape, X_scaled.shape[1])

In [ ]:
dummy_scaled = scaler.transform(dummy_flight_2[X.columns])
latent_dummy = encoder.predict(dummy_scaled)

In [ ]:
# 1. Get latent vectors for normal flights
latent_normal = encoder.predict(X_scaled)  # all normal training samples

# 2. Compute mean and covariance
mu = np.mean(latent_normal, axis=0)
cov = np.cov(latent_normal, rowvar=False) + np.eye(latent_normal.shape[1]) * 1e-6
cov_inv = np.linalg.inv(cov)

# 3. Compute latent for your test flight
latent_test = encoder.predict(scaler.transform(dummy_flight_2[X.columns]))

# 4. Mahalanobis distance per timestep
diff = latent_test - mu
mahal_distances = np.array([np.sqrt(np.dot(np.dot(d, cov_inv), d.T)) for d in diff])

dummy_flight_2["mahalanobis"] = mahal_distances



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 6)
icao_id = 'c06a7b'
df_icao = df.loc[df['icao24'] == icao_id]
df_icao = df_icao[:6]
print(df_icao)


In [ ]:
# Load the encoder
encoder = Model(
    inputs=autoencoder.input,
    outputs=autoencoder.get_layer("dense_57").output
)

# Define a function to run a sample
def run_sample(autoencoder, encoder, scaler, sample_df, features):
    
    # Scale the sample with the same function or it won't work
    sample_scaled = scaler.transform(sample_df[features])
    latent = encoder.predict(sample_scaled)
    reconstructed = autoencoder.predict(sample_scaled)
    error = np.mean(np.abs(sample_scaled - reconstructed), axis=1)
    return latent, reconstructed, error

sample_data = X.iloc[-1].to_frame().T
features = list(X.columns)

latent, reconstructed, error = run_sample(autoencoder, encoder, scaler, sample_data, features)
reconstructed_original = scaler.inverse_transform(reconstructed)
print("Latent:", latent)
print("Reconstructed:", reconstructed_original)
print("Error:", error)

In [ ]:
import numpy as np
from scipy.spatial.distance import mahalanobis
from tensorflow.keras.models import Model

df["datetime"] = pd.to_datetime(df["datetime"])

# Filter flight
icao24 = 'fb1f38'
start_time = "2022-03-28 02:26:50"
end_time = "2022-03-28 02:33:20"

flight_df = df[
    (df['icao24'] == icao24) &
    (df["datetime"] >= start_time) &
    (df["datetime"] <= end_time)
].sort_values("datetime").reset_index(drop=True)


flight_df = pd.get_dummies(flight_df, columns=["flight_phase"])

flight_df["flight_phase_Arrival"].head()

new_cols = ["flight_phase_Cruising", "flight_phase_Departure"]
flight_df[new_cols] = False

# Take last row, modify, and append
last_row = flight_df.iloc[-1].copy()
last_row["velocity"] = last_row["velocity"] + 500
last_row["datetime"] = last_row["datetime"] + pd.Timedelta(seconds=10)
flight_df = pd.concat([flight_df, last_row.to_frame().T], ignore_index=True)

# Scale the sample with the same function or it won't work
sample_scaled = scaler.transform(flight_df[features])
latent_vectors = encoder.predict(sample_scaled)

mu = np.mean(latent_vectors, axis=0)          # mean vector
cov = np.cov(latent_vectors, rowvar=False)   # covariance matrix
cov += np.eye(cov.shape[0]) * 1e-6
cov_inv = np.linalg.inv(cov)

def mahalanobis_distance(x, mean, inv_cov):
    diff = x - mean
    return np.sqrt(np.dot(np.dot(diff, inv_cov), diff.T))

mahal_distances = np.array([mahalanobis_distance(z, mu, cov_inv) for z in latent_vectors])

flight_df["mahalanobis"] = mahal_distances
print(flight_df[["velocity", "datetime", "mahalanobis"]].tail())


In [ ]:
paths_df = pd.read_csv("flight_paths.csv")
paths_df["start_time"] = pd.to_datetime(paths_df["start_time"])
paths_df["end_time"] = pd.to_datetime(paths_df["end_time"])
paths_df["duration"] = paths_df["end_time"] - paths_df["start_time"]

one_hour = pd.Timedelta(hours=1)
flights_around_one_hour = paths_df[
    (paths_df["duration"] >= one_hour) &
    (paths_df["duration"] <= one_hour + pd.Timedelta(minutes=30))  # optional ±5 min
]

# Check the result
flights_around_one_hour[["icao24", "start_time", "end_time", "duration"]]